# 第五节 运用RAG数据增强的文档问答系统

### 课程笔记大纲
1. 基于文档的问题回答系统概述
2. 构建系统的步骤
3. 文档检索和问答的实现
4. 嵌入和向量存储的作用
5. 链方法的比较

### 环境安装配置

首先，为了能够顺利进行开发工作，需要确保机器上安装了相应的Python包。

In [ ]:
%pip install --upgrade --quiet   langchain langchain-community langchainhub langchain-openai chromadb bs4

导入OpenAI模块,设置好开发密钥。

In [ ]:
import os
API_SECRET_KEY = "填写【个人资料】中获取的最新token"
BASE_URL = "https://www.ai360labs.com/openai/v1/"

os.environ["OPENAI_API_KEY"] = API_SECRET_KEY
os.environ["OPENAI_BASE_URL"] = BASE_URL

In [ ]:

import openai
from langchain_openai import ChatOpenAI,OpenAI
from langchain import hub

## 案例背景

在本指南中，我们将在Lilian Weng的[LLM Powered Autonomous Agents](https://lilianweng.github.io/posts/2023-06-23-agent/)博客文章上构建一个QA应用程序，该应用程序允许我们针对文章的内容提出问题。这是行业内第一个完整描述智能体架构的文章。

我们可以创建一个简单的索引管道和RAG链，在大约20行代码中完成此操作：

In [ ]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

## 1. 索引：加载

我们首先需要加载博客文章的内容。我们可以使用[DocumentLoaders](/docs/modules/data_connection/document_loaders/)，这些是从源加载数据并返回[Documents](https://api.python.langchain.com/en/latest/documents/langchain_core.documents.base.Document.html)列表的对象。`Document` 是一个具有`page_content`（str）和`metadata`（dict）的对象。

在这种情况下，我们将使用[WebBaseLoader](/docs/integrations/document_loaders/web_base)，它使用 `urllib` 从web URL加载HTML，并使用`BeautifulSoup`将其解析为文本。我们可以通过通过 `bs_kwargs` 传递参数来自定义HTML -> 文本解析（参见[BeautifulSoup文档](https://beautiful-soup-4.readthedocs.io/en/latest/#beautifulsoup)）。在这种情况下，只有具有类“post-content”、“post-title”或“post-header”的HTML标签是相关的，因此我们将删除所有其他标签。

In [ ]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()



查看整个博客的内容长度。

In [ ]:
len(docs[0].page_content)

打印第一个文档的内容的前500个字符

In [ ]:
print(docs[0].page_content[:500])

## 2. 索引：拆分

我们加载的文档超过42,000个字符。这太长了，无法适应许多模型的上下文窗口。即使对于那些可以将完整文章放入其上下文窗口的模型来说，模型也可能在非常长的输入中找不到信息。

为了处理这个问题，我们将把 `Document` 拆分成用于嵌入和向量存储的块。这应该有助于我们在运行时仅检索博客文章的最相关部分。

在这种情况下，我们将把文档拆分成每个块1000个字符，块之间有200个字符的重叠。重叠有助于减少将语句与与之相关的重要上下文分开的可能性。我们使用[RecursiveCharacterTextSplitter](/docs/modules/data_connection/document_transformers/recursive_text_splitter)，它将使用常见的分隔符（如换行符）递归地拆分文档，直到每个块的大小合适。这是通用文本用例的推荐文本分割器。

我们设置 `add_start_index=True`，以便将每个拆分文档在初始文档中开始的字符索引保留为元数据属性“start_index”。

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

查看切割了多少片段，66个切割后的小文本片段。

In [ ]:
len(all_splits)

查看拆分的第一个切片的内容。

In [ ]:
len(all_splits[0].page_content)

查看元数据属性。

打印后显示 {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/',
 'start_index': 7056}  索引编号是 7056

In [ ]:
all_splits[10].metadata

## 3. 索引：存储

现在我们需要索引我们的66个文本块，以便我们可以在运行时对它们进行搜索。

最常见的方法是嵌入每个文档拆分的内容，并将这些嵌入插入到一个向量数据库（或向量存储）中。

当我们想要搜索我们的拆分时，我们采取一个文本搜索查询，将其嵌入，然后执行某种“相似性”搜索，以识别与我们的查询嵌入最相似的存储拆分。

最简单的相似性度量是余弦相似度 — 我们测量每对嵌入之间的夹角的余弦（它们是高维向量）。

我们可以使用[Chroma](/docs/integrations/vectorstores/chroma)向量存储和[OpenAIEmbeddings](/docs/integrations/text_embedding/openai)模型，

在单个命令中嵌入并存储我们的所有文档拆分。

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

## 4. 检索和生成：检索

现在让我们编写实际的应用逻辑。我们想要创建一个简单的应用程序，接收用户的问题，搜索与该问题相关的文档，将检索到的文档和初始问题传递给一个模型，并返回一个答案。

首先，我们需要定义我们搜索文档的逻辑。

LangChain定义了一个[Retriever](https://python.langchain.com/docs/modules/data_connection/retrievers/)接口，该接口包装了一个索引，可以根据字符串查询返回相关的 `Documents`。

最常见的 `Retriever` 类型是[VectorStoreRetriever](https://python.langchain.com/docs/modules/data_connection/retrievers/vectorstore)，

它使用向量存储的相似性搜索能力来进行检索。任何 `VectorStore` 都可以轻松地转换为 `Retriever`，只需使用 `VectorStore.as_retriever()` 即可：

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})


In [ ]:
prompt = hub.pull("rlm/rag-prompt")
prompt

下面代码中使用了LangChain表达式（管道结构来处理数据），这是一种常见的数据处理模式。

在这个模式中，数据会按顺序通过一系列的操作，每个操作都会对数据进行一些处理，然后将结果传递给下一个操作。

- 以 `retriever | format_docs` 开始，这意味着首先应用 `retriever`，然后将其输出传递给 `format_docs` 函数。

- `format_docs` 函数接收一个文档列表（每个文档都有一个 `page_content` 属性），并将它们的内容连接起来，每两个文档之间用两个换行符分隔。

- `format_docs` 的输出然后传递给 `RunnablePassthrough()`。

- `RunnablePassthrough` 是一个类，它基本上充当恒等函数，将其输入不变地传递到输出。`RunnablePassthrough()` 的输出然后传递给 `prompt`，然后传递给 `llm`。
- `llm` 的输出传递给 `StrOutputParser()`。`StrOutputParser` 是一个类，它将输出解析为字符串。


In [ ]:

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke("有哪些类型的记忆内存?")

## 5. 检索和生成：生成

让我们将所有内容整合到一个链中，该链接收一个问题，检索相关文档，构建提示，将其传递给模型，并解析输出。

我们将使用 gpt-3.5-turbo OpenAI 聊天模型，但任何 LangChain `LLM` 或 `ChatModel` 都可以替换。

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

从hub上拉取RAG的提示词模板。

In [ ]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

打印提示词模板。

In [ ]:
example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()
example_messages

查看RAG的提示词模板。

In [ ]:
print(example_messages[0].content)

运行RAG链，它使用向量存储的相似性搜索能力来进行检索原博客文档，将搜索的结果拼接为字符串，并且作为提示词模板的`context`内容。

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

运行RAG链，询问什么是任务分解。

In [ ]:
for chunk in rag_chain.stream("What is Task Decomposition?"):
    print(chunk, end="", flush=True)

#### 自定义提示

如上所示，我们可以从提示中心加载提示（例如，[这个RAG提示](https://smith.langchain.com/hub/rlm/rag-prompt)）。

除了使用现有的RAG提示词模板，还可以使用第二节课程介绍的提示词模板技术进行自定义。


In [ ]:
from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)



运行RAG链，询问什么是任务分解。

In [ ]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

**注意**:

如果执行从hub拉取提示词模板的代码出现错误，请重新启动该笔记本，如果你本地环境有多个python版本，请尝试更换python的版本。这种错误是由于python版本所致，另外还可以升级本地的python版本解决。